# Normalize Form Fields Classifier

Thoughts:
- fill training data with forms selected based on connections based on random sampling in jur
- make the training only take place if source and labeled are different (filter on old_fields = new fields)
- don't label auto-fields label with adjacent text from another field use `_ignore_`
- consider training only within jurisdiction
- should I norm vectors in reformating function?

## Load Libraries

In [1]:
jurs = ["MI", "UT", "ME"]

In [5]:
import pandas as pd
import re
from os import walk
from sklearn.metrics import accuracy_score

if 1 == 2:
    import PyPDF2
    from sklearn.preprocessing import normalize
    import urllib.request, json
    from urllib.request import urlopen
    from bs4 import BeautifulSoup
    import os.path
    from os import path
    import numpy as np

    #!pip install py-readability-metrics
    from readability import Readability

    #!python -m nltk.downloader punkt

    import networkx as nx

    from matplotlib.pyplot import figure

    from pyvis.network import Network

    import time
    from datetime import date

    import spacy

    from sklearn.metrics import accuracy_score

    from sklearn.model_selection import KFold

    from sklearn.linear_model import LogisticRegression
    from sklearn import tree
    from sklearn.ensemble import (
        RandomForestClassifier,
        AdaBoostClassifier,
        VotingClassifier,
    )
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import GradientBoostingClassifier

    from joblib import dump, load

    import math

    from sklearn.metrics.pairwise import (
        cosine_similarity,
    )  # define matrix with all zero values

    # import nltk
    # nltk.download('stopwords')

    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize

In [2]:
import sys

sys.path.insert(0, "../../formfyxer")
import formfyxer.lit_explorer as ff

/home/brycew/Developer/LITLab/litlab_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
/home/brycew/Developer/LITLab/litlab_venv/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_lg' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/brycew/Developer/LITLab/litlab_venv/lib/python3.8/site-packages/sklearn/base.py:299: UserWarn

## Get training and test form data from each of our jurisdictions

First we'll start by grabing a set of forms with automaticly-generated form field names.

In [3]:
included_fields = [
    "users1_name",
    "users1_birthdate",
    "users1_address_line_one",
    "users1_address_line_two",
    "users1_address_city",
    "users1_address_state",
    "users1_address_zip",
    "users1_phone_number",
    "users1_email",
    "plantiffs1_name",
    "defendants1_name",
    "petitioners1_name",
    "respondents1_name",
    "docket_number",
    "trial_court_county",
    "users1_signature",
    "signature_date",
]

In [10]:
path = "ML_training/auto/"
f = []
for dirpath, dirnames, filenames in walk(path):
    f.extend(filenames)
    break
f1 = [file for file in f]

auto_df = pd.DataFrame(
    [], columns=["form_id", "jurisdiction", "group", "n", "per", "this_field"]
)

for file in f1:
    if re.search(".*\.pdf", file):
        form_id = re.search("(.*)\.pdf", file).groups()[0]
        print(path + file)
        stats = ff.parse_form(path + file, "title", "group")
        i = 0
        n_fields = len(stats["fields"])
        last_field = "null"
        for stat in stats["fields"]:
            # print(form_id,df[df["id"]==form_id]["jurisdiction"].values[0])
            # auto_df = auto_df.append(pd.DataFrame([[form_id,df[df["id"]==form_id]["jurisdiction"].values[0],df[df["id"]==form_id]["group"].values[0],i,(i/n_fields),lit.reCase(stat)]],columns=["form_id","jurisdiction","group","n","per","this_field"]), ignore_index=True,sort=False)
            auto_df = pd.concat(
                [
                    auto_df,
                    pd.DataFrame(
                        [
                            [
                                form_id,
                                "state",
                                "group",
                                i,
                                (i / n_fields),
                                ff.re_case(stat),
                            ]
                        ],
                        columns=[
                            "form_id",
                            "jurisdiction",
                            "group",
                            "n",
                            "per",
                            "this_field",
                        ],
                    ),
                ],
                ignore_index=True,
                sort=False,
            )
            i += 1

auto_df

ML_training/auto/3a09aadafc4a4732b41ad48fd313ffe8.pdf
Detecting Sentences...
Starting to find passives...
ML_training/auto/f7e6e67e408c4309b83b8a6fde22b706.pdf
Detecting Sentences...
Starting to find passives...
ML_training/auto/9dda7eb4b8b64f88abd092075ce6245d.pdf
Detecting Sentences...
Starting to find passives...
ML_training/auto/ebb2721315f845819b46cf395f956966.pdf
Detecting Sentences...
Starting to find passives...
ML_training/auto/b8dc2bf7b60946f2bbbf45cc3967f78f.pdf
Detecting Sentences...
Starting to find passives...
ML_training/auto/9ec36f386c34421b9da9f9ce6082a5ce.pdf
Detecting Sentences...
Starting to find passives...
ML_training/auto/095b9dc651ce47eb8b62e0790974970f.pdf
Detecting Sentences...
Starting to find passives...
ML_training/auto/d488e5e1f3d44d3a99c50a4ec5305af8.pdf
Detecting Sentences...
Starting to find passives...
ML_training/auto/e5e3527acd804e7581f86e14fd41f86c.pdf
Detecting Sentences...
Starting to find passives...
ML_training/auto/3902bb0b832b4fa4b20e763520101

,form_id,jurisdiction,group,n,per,this_field
0,3a09aadafc4a4732b41ad48fd313ffe8,state,group,0,0.000000,cirno
1,3a09aadafc4a4732b41ad48fd313ffe8,state,group,1,0.028571,county
2,3a09aadafc4a4732b41ad48fd313ffe8,state,group,2,0.057143,*docket number
3,3a09aadafc4a4732b41ad48fd313ffe8,state,group,3,0.085714,casename
4,3a09aadafc4a4732b41ad48fd313ffe8,state,group,4,0.114286,adoptname
...,...,...,...,...,...,...
300,3902bb0b832b4fa4b20e7635201017aa,state,group,33,0.868421,*users1 signature 2
301,3902bb0b832b4fa4b20e7635201017aa,state,group,34,0.894737,printed name 2
302,3902bb0b832b4fa4b20e7635201017aa,state,group,35,0.921053,request order respondent 3
303,3902bb0b832b4fa4b20e7635201017aa,state,group,36,0.947368,approved district court 3


In [11]:
auto_df["this_field"].unique()

array(['cirno', 'county', '*docket number', 'casename', 'adoptname',
       '*users1 birthdate', 'presentname', 'spname', 'bpname',
       'adopteename', 'npname', 'npaddress', 'npcity', 'nptelno',
       'aspname', 'aspaddress', 'aspcity', 'asptelno', 'pdate  1',
       'paddress', 'pcity', 'ptelno', 'pdate  2', 'bpaddress', 'bpcity',
       'bptelno', 'aname', 'abarno', 'aaddress', 'acity', 'atelno',
       'check box', 'text  1', 'text  2', 'text  3', 'undefined  1',
       '*users1 name', '*users1 address line one', 'city state zip',
       '*users1 phone number', 'email', 'undefined  2',
       'juvenile court utah', 'judicial district', 'state utah interest',
       'last name first', 'utah code', 'case number', 'incidents',
       'parents names', 'request judge waive requirement  1',
       'request judge waive requirement  2',
       'request judge waive requirement  3', 'approved court judges  1',
       'petition court records  1', 'page  1',
       'request court requiremen

Now we'll grab the same set of forms with ground truth label names. 

In [12]:
path = "ML_training/ground/"
f = []
for dirpath, dirnames, filenames in walk(path):
    f.extend(filenames)
    break
f1 = [file for file in f]

ground_df = pd.DataFrame(
    [], columns=["form_id", "n"] + included_fields + ["this_field_ground"]
)

for file in f1:
    if re.search(".*\.pdf", file):
        form_id = re.search("(.*)\.pdf", file).groups()[0]
        print(path + file)
        stats = ff.parse_form(path + file, "title", "group")
        i = 0
        n_fields = len(stats["fields"])
        last_field = "null"
        for stat in stats["fields"]:
            # print("\t"+str(i)+","+str(i/n_fields)+","+last_field+","+stat[:5])
            if "_ignore_" not in stat:
                #
                # this * is a hold over from form explorer that should be removed
                if re.sub("^\*", "", stat) not in included_fields:
                    stat = "_nonstandard_"
                field_matches = []
                for item in included_fields:
                    if last_field == item:
                        field_matches.append(1)
                    else:
                        field_matches.append(0)
                ground_df = pd.concat(
                    [
                        ground_df,
                        pd.DataFrame(
                            [[form_id, i] + field_matches + [stat]],
                            columns=["form_id", "n"]
                            + included_fields
                            + ["this_field_ground"],
                        ),
                    ],
                    ignore_index=True,
                    sort=False,
                )
            last_field = stat
            i += 1

ground_df

ML_training/ground/3a09aadafc4a4732b41ad48fd313ffe8.pdf
Detecting Sentences...
Starting to find passives...
ML_training/ground/f7e6e67e408c4309b83b8a6fde22b706.pdf
Detecting Sentences...
Starting to find passives...
ML_training/ground/9dda7eb4b8b64f88abd092075ce6245d.pdf
Detecting Sentences...
Starting to find passives...
ML_training/ground/ebb2721315f845819b46cf395f956966.pdf
Detecting Sentences...
Starting to find passives...
ML_training/ground/b8dc2bf7b60946f2bbbf45cc3967f78f.pdf
Detecting Sentences...
Starting to find passives...
ML_training/ground/9ec36f386c34421b9da9f9ce6082a5ce.pdf
Detecting Sentences...
Starting to find passives...
ML_training/ground/095b9dc651ce47eb8b62e0790974970f.pdf
Detecting Sentences...
Starting to find passives...
ML_training/ground/d488e5e1f3d44d3a99c50a4ec5305af8.pdf
Detecting Sentences...
Starting to find passives...
ML_training/ground/e5e3527acd804e7581f86e14fd41f86c.pdf
Detecting Sentences...
Starting to find passives...
ML_training/ground/3902bb0b8

,form_id,n,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,users1_address_state,users1_address_zip,users1_phone_number,users1_email,plantiffs1_name,defendants1_name,petitioners1_name,respondents1_name,docket_number,trial_court_county,users1_signature,signature_date,this_field_ground
0,3a09aadafc4a4732b41ad48fd313ffe8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
1,3a09aadafc4a4732b41ad48fd313ffe8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
2,3a09aadafc4a4732b41ad48fd313ffe8,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
3,3a09aadafc4a4732b41ad48fd313ffe8,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
4,3a09aadafc4a4732b41ad48fd313ffe8,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,3902bb0b832b4fa4b20e7635201017aa,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
278,3902bb0b832b4fa4b20e7635201017aa,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
279,3902bb0b832b4fa4b20e7635201017aa,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_
280,3902bb0b832b4fa4b20e7635201017aa,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,_nonstandard_


In [13]:
ground_df["this_field_ground"].unique()

array(['_nonstandard_', '*docket_number', '*trial_court_county',
       '*users1_birthdate', '*users1_address_line_one',
       '*users1_signature', '*signature_date', '*users1_name',
       '*users1_address_line_two', '*users1_email',
       '*users1_phone_number', '*users1_address_city',
       '*users1_address_state', '*users1_address_zip',
       '*petitioners1_name', '*respondents1_name'], dtype=object)

## All together now

Let's merge the form data and create some features. 

In [14]:
df_all = auto_df.merge(ground_df, how="inner")
print(len(df_all))
display(df_all.head())

# take care of some categorical variables
parts = pd.get_dummies(df_all["jurisdiction"])
parts = pd.concat([parts, pd.get_dummies(df_all["group"])], axis=1)
df_all = pd.concat([df_all, parts], axis=1)
jurisdictions = df_all["jurisdiction"].unique().tolist()

# groups = df_all['group'].unique().tolist()
groups = ["state"]

df_all = df_all.drop(["jurisdiction", "group"], 1, errors="ignore").copy()
display(df_all.head())
df_all["this_field_vec"] = df_all["this_field"].apply(ff.vectorize)
# df_all["last_field_ground_vec"] = df_all["last_field_ground"].apply(vectorize)
df_all = df_all.drop(["this_field", "last_field_ground"], 1, errors="ignore").copy()
display(df_all.head())
# df_all["last_field_ground_vec"] = df_all["last_field_ground_vec"].apply(norm)
display(df_all.head())
df_all = pd.concat(
    [
        df_all,
        pd.DataFrame(
            df_all.this_field_vec.values.tolist(), index=df_all.index
        ).add_prefix("vec_this"),
    ],
    axis=1,
)
# df_all = pd.concat([df_all, pd.DataFrame(df_all.last_field_ground_vec.values.tolist(), index= df_all.index).add_prefix("vec_last")], axis=1)
# df_all = df_all.drop(['this_field_vec','last_field_ground_vec'], 1, errors='ignore').copy()
df_all = df_all.drop(["this_field_vec"], 1, errors="ignore").copy()
display(df_all.head())

280


,form_id,jurisdiction,group,n,per,this_field,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,...,users1_email,plantiffs1_name,defendants1_name,petitioners1_name,respondents1_name,docket_number,trial_court_county,users1_signature,signature_date,this_field_ground
0,3a09aadafc4a4732b41ad48fd313ffe8,state,group,0,0.000000,cirno,0,0,0,0,...,0,0,0,0,0,0,0,0,0,_nonstandard_
1,3a09aadafc4a4732b41ad48fd313ffe8,state,group,1,0.028571,county,0,0,0,0,...,0,0,0,0,0,0,0,0,0,_nonstandard_
2,3a09aadafc4a4732b41ad48fd313ffe8,state,group,2,0.057143,*docket number,0,0,0,0,...,0,0,0,0,0,0,0,0,0,_nonstandard_
3,3a09aadafc4a4732b41ad48fd313ffe8,state,group,3,0.085714,casename,0,0,0,0,...,0,0,0,0,0,0,0,0,0,_nonstandard_
4,3a09aadafc4a4732b41ad48fd313ffe8,state,group,4,0.114286,adoptname,0,0,0,0,...,0,0,0,0,0,0,0,0,0,_nonstandard_


/tmp/ipykernel_1354069/1703278981.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_all = df_all.drop(["jurisdiction", "group"], 1, errors="ignore").copy()


,form_id,n,per,this_field,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,users1_address_state,...,plantiffs1_name,defendants1_name,petitioners1_name,respondents1_name,docket_number,trial_court_county,users1_signature,signature_date,this_field_ground,state
0,3a09aadafc4a4732b41ad48fd313ffe8,0,0.000000,cirno,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,_nonstandard_,1
1,3a09aadafc4a4732b41ad48fd313ffe8,1,0.028571,county,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,_nonstandard_,1
2,3a09aadafc4a4732b41ad48fd313ffe8,2,0.057143,*docket number,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,_nonstandard_,1
3,3a09aadafc4a4732b41ad48fd313ffe8,3,0.085714,casename,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,_nonstandard_,1
4,3a09aadafc4a4732b41ad48fd313ffe8,4,0.114286,adoptname,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,_nonstandard_,1


/tmp/ipykernel_1354069/1703278981.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_all = df_all.drop(["this_field", "last_field_ground"], 1, errors="ignore").copy()


,form_id,n,per,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,users1_address_state,users1_address_zip,...,defendants1_name,petitioners1_name,respondents1_name,docket_number,trial_court_county,users1_signature,signature_date,this_field_ground,state,this_field_vec
0,3a09aadafc4a4732b41ad48fd313ffe8,0,0.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,3a09aadafc4a4732b41ad48fd313ffe8,1,0.028571,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[-0.08368088617093719, -0.03452417496875871, -..."
2,3a09aadafc4a4732b41ad48fd313ffe8,2,0.057143,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.03083843926583192, 0.008258316422247608, 0...."
3,3a09aadafc4a4732b41ad48fd313ffe8,3,0.085714,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,3a09aadafc4a4732b41ad48fd313ffe8,4,0.114286,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


,form_id,n,per,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,users1_address_state,users1_address_zip,...,defendants1_name,petitioners1_name,respondents1_name,docket_number,trial_court_county,users1_signature,signature_date,this_field_ground,state,this_field_vec
0,3a09aadafc4a4732b41ad48fd313ffe8,0,0.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,3a09aadafc4a4732b41ad48fd313ffe8,1,0.028571,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[-0.08368088617093719, -0.03452417496875871, -..."
2,3a09aadafc4a4732b41ad48fd313ffe8,2,0.057143,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.03083843926583192, 0.008258316422247608, 0...."
3,3a09aadafc4a4732b41ad48fd313ffe8,3,0.085714,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,3a09aadafc4a4732b41ad48fd313ffe8,4,0.114286,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,_nonstandard_,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


/tmp/ipykernel_1354069/1703278981.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_all = df_all.drop(["this_field_vec"], 1, errors="ignore").copy()


,form_id,n,per,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,users1_address_state,users1_address_zip,...,vec_this290,vec_this291,vec_this292,vec_this293,vec_this294,vec_this295,vec_this296,vec_this297,vec_this298,vec_this299
0,3a09aadafc4a4732b41ad48fd313ffe8,0,0.000000,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3a09aadafc4a4732b41ad48fd313ffe8,1,0.028571,0,0,0,0,0,0,0,...,-0.024812,0.093562,-0.047287,0.049727,-0.046225,0.101842,-0.002813,-0.087645,0.057705,0.046242
2,3a09aadafc4a4732b41ad48fd313ffe8,2,0.057143,0,0,0,0,0,0,0,...,-0.017769,0.004765,0.043737,-0.004850,0.014582,0.096194,0.011439,-0.062137,0.023260,0.011969
3,3a09aadafc4a4732b41ad48fd313ffe8,3,0.085714,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,3a09aadafc4a4732b41ad48fd313ffe8,4,0.114286,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
df_class = pd.concat([df_all[["form_id"]], df_all[jurisdictions]], axis=1)
df_class = pd.concat([df_class, df_all[["n", "per"]]], axis=1)
df_class = pd.concat([df_class, df_all[groups], df_all[included_fields]], axis=1)
df_class = pd.concat(
    [
        df_class,
        df_all[[x for x in df_all.columns if "vec_" in x]],
        df_all[["this_field_ground"]],
    ],
    axis=1,
)
df_class

,form_id,state,n,per,state,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,...,vec_this291,vec_this292,vec_this293,vec_this294,vec_this295,vec_this296,vec_this297,vec_this298,vec_this299,this_field_ground
0,3a09aadafc4a4732b41ad48fd313ffe8,1,0,0.000000,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,_nonstandard_
1,3a09aadafc4a4732b41ad48fd313ffe8,1,1,0.028571,1,0,0,0,0,0,...,0.093562,-0.047287,0.049727,-0.046225,0.101842,-0.002813,-0.087645,0.057705,0.046242,_nonstandard_
2,3a09aadafc4a4732b41ad48fd313ffe8,1,2,0.057143,1,0,0,0,0,0,...,0.004765,0.043737,-0.004850,0.014582,0.096194,0.011439,-0.062137,0.023260,0.011969,_nonstandard_
3,3a09aadafc4a4732b41ad48fd313ffe8,1,3,0.085714,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,_nonstandard_
4,3a09aadafc4a4732b41ad48fd313ffe8,1,4,0.114286,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,_nonstandard_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,3902bb0b832b4fa4b20e7635201017aa,1,31,0.815789,1,0,0,0,0,0,...,-0.000230,-0.024969,0.020514,0.004731,-0.034679,0.087214,-0.083711,-0.006410,0.069600,_nonstandard_
276,3902bb0b832b4fa4b20e7635201017aa,1,32,0.842105,1,0,0,0,0,0,...,0.000806,0.030701,0.022904,0.025957,0.008180,0.091074,-0.031061,0.002454,0.037469,_nonstandard_
277,3902bb0b832b4fa4b20e7635201017aa,1,33,0.868421,1,0,0,0,0,0,...,-0.007429,0.036310,0.009307,0.045329,0.022455,0.087298,-0.025626,-0.006203,0.034363,_nonstandard_
278,3902bb0b832b4fa4b20e7635201017aa,1,34,0.894737,1,0,0,0,0,0,...,0.002720,0.038181,0.054661,0.014350,-0.013956,0.120650,-0.054173,0.023242,-0.002885,_nonstandard_


In [16]:
# down sample
if 1 == 1:
    df0 = df_class[df_class["this_field_ground"] != "_nonstandard_"]
    df1 = df_class[df_class["this_field_ground"] == "_nonstandard_"][
        : len(df_class[df_class["this_field_ground"] != "_nonstandard_"])
    ]
    df_class = pd.concat([df0, df1], ignore_index=True)

    display(df_class)

,form_id,state,n,per,state,users1_name,users1_birthdate,users1_address_line_one,users1_address_line_two,users1_address_city,...,vec_this291,vec_this292,vec_this293,vec_this294,vec_this295,vec_this296,vec_this297,vec_this298,vec_this299,this_field_ground
0,3a09aadafc4a4732b41ad48fd313ffe8,1,28,0.800000,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,*docket_number
1,3a09aadafc4a4732b41ad48fd313ffe8,1,29,0.828571,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,*trial_court_county
2,3a09aadafc4a4732b41ad48fd313ffe8,1,30,0.857143,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,*users1_birthdate
3,3a09aadafc4a4732b41ad48fd313ffe8,1,31,0.885714,1,0,0,0,0,0,...,0.039622,0.066307,-0.037217,-0.042500,0.060465,-0.028172,-0.006263,-0.007361,-0.032218,*users1_address_line_one
4,f7e6e67e408c4309b83b8a6fde22b706,1,13,0.325000,1,0,0,0,0,0,...,-0.014465,0.032156,0.018218,-0.056605,0.030599,0.057154,-0.105767,0.019321,0.065500,*docket_number
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,f7e6e67e408c4309b83b8a6fde22b706,1,20,0.500000,1,0,0,0,0,0,...,0.045047,-0.022642,0.044735,0.068387,0.060183,0.081443,-0.093463,-0.012765,0.006185,_nonstandard_
98,f7e6e67e408c4309b83b8a6fde22b706,1,21,0.525000,1,0,0,0,0,0,...,0.031939,-0.037314,0.038870,0.020290,0.060802,0.068418,-0.079105,-0.024666,0.079737,_nonstandard_
99,f7e6e67e408c4309b83b8a6fde22b706,1,22,0.550000,1,0,0,0,0,0,...,-0.036939,-0.009072,0.046844,0.003468,0.025814,0.084823,-0.081481,-0.040286,0.067102,_nonstandard_
100,f7e6e67e408c4309b83b8a6fde22b706,1,23,0.575000,1,0,0,0,0,0,...,-0.037373,-0.016221,0.051468,-0.003301,0.018772,0.083066,-0.070632,-0.021455,0.070191,_nonstandard_


In [17]:
1 - len(df_class["this_field_ground"].unique()) / len(df_class["this_field_ground"])

0.8431372549019608

In [18]:
form_list = pd.DataFrame([], columns=["form_id"])
form_list["form_id"] = df_class["form_id"].unique()
holdout = form_list.sample(frac=0.2)

# create a dataframe containing a random sample of rows
class_holdout = df_class.loc[df_class.form_id.isin(holdout["form_id"])]

# create a dataframe that conatins the rows from except for those in holdout
class_training = df_class.loc[~df_all.form_id.isin(holdout["form_id"])]

# make a training dataframe containing just features
features_train = class_training.drop(["form_id", "this_field_ground"], axis=1).values

# make a training dataframe containing only the target
labels_train = class_training["this_field_ground"].values

# make a testing/holdout dataframe containing just features
features_test = class_holdout.drop(["form_id", "this_field_ground"], axis=1).values

# make a testing/holdout dataframe containing only the target
labels_test = class_holdout["this_field_ground"].values

print("size of training", len(class_training))
print("size of testing", len(class_holdout))

size of training 69
size of testing 37


In [19]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(fit_intercept=False, C=1e9)
clf_1 = model.fit(features_train, labels_train)
pred = clf_1.predict(features_test)
prob = clf_1.predict_proba(features_test)[:, 1]
print("Logistic Regression")

print(accuracy_score(labels_test, pred))
# evaluate(prob, pred, labels_test)

print("")
print(labels_test, "\n")
print(pred)

Logistic Regression
0.7837837837837838

['*docket_number' '*trial_court_county' '*users1_birthdate'
 '*users1_address_line_one' '*users1_address_line_one' '*docket_number'
 '*trial_court_county' '*users1_name' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_'] 

['_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '*users1_birthdate' '*users1_address_line_two' '*users1_address_line_two'
 '*users1_phone_number' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nons

/home/brycew/Developer/LITLab/litlab_venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
from sklearn.ensemble import AdaBoostClassifier

clf_2 = AdaBoostClassifier()
clf_2 = clf_2.fit(features_train, labels_train)
pred = clf_2.predict(features_test)
prob = clf_2.predict_proba(features_test)  # [:,1]
print("AdaBoost Forest")

print(accuracy_score(labels_test, pred))
# evaluate(prob,pred, labels_test)

print("")
print(labels_test, "\n")
print(pred)

AdaBoost Forest
0.16216216216216217

['*docket_number' '*trial_court_county' '*users1_birthdate'
 '*users1_address_line_one' '*users1_address_line_one' '*docket_number'
 '*trial_court_county' '*users1_name' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_'] 

['*users1_phone_number' '*users1_phone_number' '*users1_phone_number'
 '_nonstandard_' '*users1_birthdate' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '*users1_phone_number' '_nonstandard_' '_nonstandard_'
 '*users1_phone_number' '*users1_phone_number' '_nonstandard_'
 '*users1_phone_number' '*users1_phone_number

In [21]:
from sklearn.ensemble import GradientBoostingClassifier

clf_3 = GradientBoostingClassifier()
clf_3 = clf_3.fit(features_train, labels_train)
pred = clf_3.predict(features_test)
prob = clf_3.predict_proba(features_test)  # [:,1]
print("Gradient Boost")

print(accuracy_score(labels_test, pred))
# evaluate(prob,pred, labels_test)

print("")
print(labels_test, "\n")
print(pred)

Gradient Boost
0.7837837837837838

['*docket_number' '*trial_court_county' '*users1_birthdate'
 '*users1_address_line_one' '*users1_address_line_one' '*docket_number'
 '*trial_court_county' '*users1_name' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_'] 

['_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '*users1_birthdate' '_nonstandard_' '*users1_name'
 '*users1_address_line_two' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard

In [22]:
from sklearn.ensemble import RandomForestClassifier

clf_4 = RandomForestClassifier()
clf_4 = clf_4.fit(features_train, labels_train)
pred = clf_4.predict(features_test)
prob = clf_4.predict_proba(features_test)  # [:,1]
print("Random Forest")

print(accuracy_score(labels_test, pred))
# evaluate(prob,pred, labels_test)

print("")
print(labels_test, "\n")
print(pred)

Random Forest
0.7837837837837838

['*docket_number' '*trial_court_county' '*users1_birthdate'
 '*users1_address_line_one' '*users1_address_line_one' '*docket_number'
 '*trial_court_county' '*users1_name' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_'] 

['_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '*users1_birthdate' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstandard_'
 '_nonstandard_' '_nonstandard_' '_nonstandard_' '_nonstan

In [24]:
from joblib import dump, load

overwrite = input("Overwrite old data? (y/n)\n")
if "y" in overwrite.lower():
    dump(included_fields, "../formfyxer/data/included_fields.joblib")
    dump(clf_4, "../formfyxer/data/clf_field_names.joblib")
    dump(jurisdictions, "../formfyxer/data/jurisdictions.joblib")
    dump(groups, "../formfyxer/data/groups.joblib")